# Réarrengement data Solordinario

In [2]:
import numpy as np
import glob 
import os.path
import scipy.io.wavfile as wavfile

In [4]:
path = 'data'

def listdirectory(path): 
    fichier=[] 
    l = glob.glob(path+'\\*') 
    for i in l: 
        if os.path.isdir(i): fichier.extend(listdirectory(i)) 
        else: fichier.append(i) 
    return fichier

files = listdirectory(path)

In [13]:
long = 3 #durée à conserver en sec
for file in files:
    Fe, sig = wavfile.read(file)
    new_dir = 'data_2/'+file[5:]
    wavfile.write(new_dir, Fe, sig[:long*Fe])

# Dictionnaire avec Pitch et Velocite

In [32]:
import descriptors
import torch
import numpy as np

In [95]:
path = 'data_2'
files = listdirectory(path)
nb_files = len(files)
print(nb_files)
sample_per_sec = 250

dict_env = {'file_name' : [[]]*nb_files, 'env' : [[]]*nb_files, 'vel' : [[]]*nb_files, 'pitch' : [[]]*nb_files, 'corde' : [[]]*nb_files}
f = 0

for file in files:
    name = file[7:]
    vel = file[14:16]
    corde = file[17:19]
    if file[-6]=='#':
        pitch = file[-7:-4]
    else :
        pitch = file[-6:-4]
        
    Fe, sig = wavfile.read(file)
    sig = sig.astype(np.float32)
    env = np.zeros([2,long*sample_per_sec])
    env[0,:] = descriptors.extract_loudness(sig, sampling_rate=Fe, block_size=int(Fe/250), n_fft=2048)[:long*sample_per_sec]
    env[1,:] = descriptors.extract_pitch(sig, sampling_rate=Fe, block_size=int(Fe/250))[:long*sample_per_sec]
    
    dict_env['file_name'][f] = file
    dict_env['env'][f] = env
    dict_env['vel'][f] = vel
    dict_env['pitch'][f] = tone
    dict_env['corde'][f] = corde
    print(f)
    f += 1
  

279


KeyboardInterrupt: 

In [93]:
# Save
np.save('data_dict.npy', dict_env) 

# Load
# dict_r = np.load('data_dict.npy').item() 

### NB : si quelqu'un arrive à faire tourner ça vite et peux envoyer le fichier 'data_dict.npy', ça serait cool

# Création dataset pour torch (pas encore fait, les codes ici sont des copiés/collés, mais je n'ai pas encore travaillé dessus)

In [ ]:
import os
from torch.utils.data import Dataset


In [ ]:
class TESNamesDataset(Dataset):
    def __init__(self, data_root):
        self.samples = []

        for race in os.listdir(data_root):
            race_folder = os.path.join(data_root, race)

            for gender in os.listdir(race_folder):
                gender_filepath = os.path.join(race_folder, gender)

                with open(gender_filepath, 'r') as gender_file:
                    for name in gender_file.read().splitlines():
                        self.samples.append((race, gender, name))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]


if __name__ == '__main__':
    dataset = TESNamesDataset('/home/syafiq/Data/tes-names/')
    print(len(dataset))
    print(dataset[420])